In [1]:
TRAIN_SCRIPT_ALPACA = "/home/omadbek/projects/alpaca/train.py"
# weights of alpaca
BASE_MODEL_ALPACA_PATH = (
    "/home/omadbek/.cache/huggingface/hub/"
    "models--chavinlo--alpaca-native/"
    "snapshots/3bf09cbff2fbd92d7d88a0f70ba24fca372befdf"
)

CUSTOM_DATA_PATH = "/home/omadbek/projects/ArcheType/fine_tuned_data/full_train_finetune.json"

FINE_TUNED_WEIGHTS_PATH = "/home/omadbek/projects/alpaca/outputs"

PAPERMILL_ROOT = "/home/omadbek/projects/ArcheType/papermill_notebooks"

MODEL_NAME = "flan-t5-xxl" #llama, flan-ul2, flan-t5, flan-t5-xxl", alpaca-fine-tuned

SHERLOCK_PATH  = "/home/omadbek/projects/Sherlock"

In [2]:
import os
import json
import subprocess
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from pathlib import Path
import shutil

In [3]:
# Setting up CUDA devices for training and inference
training_env = os.environ.copy()
training_env["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"

inference_env = os.environ.copy()
inference_env["CUDA_VISIBLE_DEVICES"] = "6,7"

In [4]:
TRAIN_ARGS = [
    "torchrun",
    "--nproc_per_node=4",
    "--rdzv_backend=c10d",
    "--rdzv_endpoint=127.0.0.1:29500",
    TRAIN_SCRIPT_ALPACA,
    "--model_name_or_path", BASE_MODEL_ALPACA_PATH,
    "--data_path", CUSTOM_DATA_PATH,
    "--bf16", "True",
    "--per_device_train_batch_size", "2",
    "--per_device_eval_batch_size", "2",
    "--gradient_accumulation_steps", "16",
    "--eval_strategy", "no",
    "--save_strategy", "steps",
    "--save_steps", "2000",
    "--save_total_limit", "1",
    "--weight_decay", "0.0",
    "--warmup_ratio", "0.03",
    "--lr_scheduler_type", "cosine",
    "--logging_steps", "1",
    "--fsdp", "full_shard auto_wrap",
    "--fsdp_transformer_layer_cls_to_wrap", "LlamaDecoderLayer",
    "--tf32", "True",
    "--overwrite_output_dir",
]

In [5]:
if MODEL_NAME == "alpaca-fine-tuned":
    # Model fine-tuning
    BEST_EPOCHS = 10
    BEST_LR      = 2e-5
    best_name    = f"e{BEST_EPOCHS}_lr{BEST_LR:.0e}"
    CONDA_BIN = "/opt/anaconda3/condabin/conda"

    train_cmd = (
            [CONDA_BIN, "run", "-n", "alpaca"]
            + TRAIN_ARGS
            + [
                "--num_train_epochs", str(BEST_EPOCHS),
                "--learning_rate",      str(BEST_LR),
                "--output_dir",         FINE_TUNED_WEIGHTS_PATH,
                "--run_name",           best_name,
            ]
    )
    print(f"\n→ TRAIN {best_name}")
    subprocess.run(train_cmd, check=True, env=training_env)

In [7]:
# Model inferencing
N_RUNS = 10
inference_results = []
CONDA_BIN = "/opt/anaconda3/condabin/conda"

for run_idx in range(1, N_RUNS + 1):
    run_id = f"{MODEL_NAME}_run{run_idx}"
    pm_cmd = [
        CONDA_BIN, "run", "-n", "archetype",
        "--no-capture-output",
        "papermill",
        "custom_inference.ipynb",
        f"{PAPERMILL_ROOT}/test_out_{run_id}.ipynb",
        "-p", "tune", run_id,
        "-p", "model_name", MODEL_NAME,
        "-p", "sherlock_path", SHERLOCK_PATH,
        "-p", "prompt_type", "chain-thoughts", #zero-shot, few-shot, chain-thoughts
    ]
    print(f"→ INFERENCE {run_id}")
    subprocess.run(pm_cmd, check=True, env=inference_env)


→ INFERENCE flan-t5-xxl_run1


/home/omadbek/.conda/envs/archetype/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Input Notebook:  custom_inference.ipynb
Output Notebook: /home/omadbek/projects/ArcheType/papermill_notebooks/test_out_flan-t5-xxl_run1.ipynb
Executing: 100%|██████████| 44/44 [00:53<00:00,  1.23s/cell]


→ INFERENCE flan-t5-xxl_run2


/home/omadbek/.conda/envs/archetype/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Input Notebook:  custom_inference.ipynb
Output Notebook: /home/omadbek/projects/ArcheType/papermill_notebooks/test_out_flan-t5-xxl_run2.ipynb
Executing: 100%|██████████| 44/44 [01:01<00:00,  1.40s/cell]


→ INFERENCE flan-t5-xxl_run3


...

/home/omadbek/.conda/envs/archetype/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Input Notebook:  custom_inference.ipynb
Output Notebook: /home/omadbek/projects/ArcheType/papermill_notebooks/test_out_flan-t5-xxl_run8.ipynb
Executing: 100%|██████████| 44/44 [00:57<00:00,  1.30s/cell]


→ INFERENCE flan-t5-xxl_run9


/home/omadbek/.conda/envs/archetype/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Input Notebook:  custom_inference.ipynb
Output Notebook: /home/omadbek/projects/ArcheType/papermill_notebooks/test_out_flan-t5-xxl_run9.ipynb
Executing: 100%|██████████| 44/44 [00:54<00:00,  1.25s/cell]


→ INFERENCE flan-t5-xxl_run10


/home/omadbek/.conda/envs/archetype/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Input Notebook:  custom_inference.ipynb
Output Notebook: /home/omadbek/projects/ArcheType/papermill_notebooks/test_out_flan-t5-xxl_run10.ipynb
Executing: 100%|██████████| 44/44 [00:59<00:00,  1.36s/cell]


In [7]:
# Hyperparameter tuning of the alpaca-fine-tuned model
"""
#  Your hyperparameter grid
GRID = [
    (3, 1e-5),
    (5, 1e-5),
    (8, 1e-5),
    (10, 1e-5),
    (15, 1e-5),
    (18, 1e-5),
    (20, 1e-5),
    (3, 2e-5),
    (5, 2e-5),
    (8, 2e-5),
    (10, 2e-5),
    (15, 2e-5),
    (18, 2e-5),
    (20, 2e-5),
]

results = []
CONDA_BIN = "/opt/anaconda3/condabin/conda"

for epochs, lr in GRID:
    name   = f"epoch{epochs}_lr{lr:.0e}"

    # ────────────────────────────────────────────────────────────

    # 1) TRAIN under alpaca env
    train_cmd = (
        [CONDA_BIN, "run", "-n", "alpaca"]
        + TRAIN_ARGS
        + [
            "--num_train_epochs", str(epochs),
            "--learning_rate",      str(lr),
            "--output_dir",         FINE_TUNED_WEIGHTS_PATH,
            "--run_name",           name,
        ]
    )
    print(f"\n→ TRAIN {name}")
    result = subprocess.run(train_cmd, check=True, env=training_env)


    # ────────────────────────────────────────────────────────────
    # 2) INFERENCE via Papermill under archetype env
    pm_cmd = [
        CONDA_BIN, "run", "-n", "archetype",
        "--no-capture-output",
        "papermill",
        "custom_inference.ipynb",
        f"{PAPERMILL_ROOT}/test_out_{name}.ipynb",
        "-p", "tune", str(name),
        "-p", "model_name", "alpaca-fine-tuned",
        "-p" "sherlock_path", SHERLOCK_PATH,
    ]
    print(f"→ INFERENCE {name}")
    subprocess.run(pm_cmd, check=True, env=inference_env)

# ────────────────────────────────────────────────────────────────
print("\n=== EVERYTHING IS DONE! ===")
"""

'\n#  Your hyperparameter grid\nGRID = [\n    (3, 1e-5),\n    (5, 1e-5),\n    (8, 1e-5),\n    (10, 1e-5),\n    (15, 1e-5),\n    (18, 1e-5),\n    (20, 1e-5),\n    (3, 2e-5),\n    (5, 2e-5),\n    (8, 2e-5),\n    (10, 2e-5),\n    (15, 2e-5),\n    (18, 2e-5),\n    (20, 2e-5),\n]\n\nresults = []\nCONDA_BIN = "/opt/anaconda3/condabin/conda"\n\nfor epochs, lr in GRID:\n    name   = f"epoch{epochs}_lr{lr:.0e}"\n\n    # ────────────────────────────────────────────────────────────\n\n    # 1) TRAIN under alpaca env\n    train_cmd = (\n        [CONDA_BIN, "run", "-n", "alpaca"]\n        + TRAIN_ARGS\n        + [\n            "--num_train_epochs", str(epochs),\n            "--learning_rate",      str(lr),\n            "--output_dir",         FINE_TUNED_WEIGHTS_PATH,\n            "--run_name",           name,\n        ]\n    )\n    print(f"\n→ TRAIN {name}")\n    result = subprocess.run(train_cmd, check=True, env=training_env)\n\n\n    # ────────────────────────────────────────────────────────────\